# Finviz Analytics


### What is Finviz?
FinViz aims to make market information accessible and provides a lot of data in visual snapshots, allowing traders and investors to quickly find the stock, future or forex pair they are looking for. The site provides advanced screeners, market maps, analysis, comparative tools and charts.

### Why?
Leverage NRT financial stats to create custom stock screens and perspectives to identify value with in volatile market conditions.

***
## Prerequisites 

### finviz
Finviz is a stock screener and trading tool used for creating financial displays. Professional traders frequently use this platform to save time because Finviz allows traders and investors to quickly screen and find stocks based on set criteria.


### pandas, pandas_profiling
Pandas needs no introduction.  Pandas_profiling creates beautiful html data profiles.

### nest_asyncio
Nest_asyncio supports asynchronous call for use with an interactive broker.



In [2]:
import pandas as pd 
import logging 
from finviz.screener import Screener

log = logging.getLogger() 
console = logging.StreamHandler()
format_str = '%(asctime)s\t%(levelname)s -- %(processName)s -- %(message)s'
console.setFormatter(logging.Formatter(format_str))
log.addHandler(console) 
log.setLevel(logging.INFO) 


In [6]:
#from matplotlib.backends.backend_pdf import PdfPages
#!pip install weasyprint
#!pip install cairo
#import weasyprint
#!pip install pdfkit

***
## Load environment and runtime variables



In [15]:
'''
MODULE is used to identify and segment runtime and environment variables from config files.
'''
MODULE = 'ldg_finviz'


'''
Load configuration fiiles from \config.  Instansiate variables with config file names.  
'''
import os 
d = os.getcwd()
df = d + '\\config\\'
try: 
    for i in os.listdir(df):
        k = i[:-4] 
        v = open(df + i).read()
        v = eval(v)
        exec("%s=%s" % (k,v))   
        log.info('loaded: ' + k)
except:
    log.error('issue encountered with eval(data): ' + str(v))


2020-06-30 17:37:50,809	INFO -- MainProcess -- loaded: api_params
2020-06-30 17:37:50,856	INFO -- MainProcess -- loaded: data_var
2020-06-30 17:37:50,857	INFO -- MainProcess -- loaded: env_var
2020-06-30 17:37:50,858	INFO -- MainProcess -- loaded: err_var
2020-06-30 17:37:50,859	INFO -- MainProcess -- loaded: map_generic_fn
2020-06-30 17:37:50,860	INFO -- MainProcess -- loaded: map_landing_dataset_code
2020-06-30 17:37:50,878	INFO -- MainProcess -- loaded: transform


In [34]:
def get_data_finviz(generate_data_profile=False):
    '''
    Download FinViz 15min delayed stock data.  
    * filter - Filter stock universe using filters variable.  
    * Select datasets to download using the map_api_fv_table_key.config  
    * Dataset options include: 
        'Overview': '111',
        'Valuation': '121',
        'Ownership': '131',
        'Performance': '141',
        'Custom': '152',
        'Financial': '161',
        'Technical': '171'
    * Refer to /docs for dataset details.
    
    Output data in .csv format to landing.
    '''
    import nest_asyncio
    nest_asyncio.apply()
    
    #load variables
    ldg_path = env_var.get('ldg_path') 
    filters = api_params['filter']

    #loop through datasets to download from Screener & write to file.
    for i in api_params['datasets']:
        log.info('downloading:' + i.get('dataset').lower())
        stock_list = Screener(filters=filters, table= i.get('dataset'))
        stock_list.to_csv(ldg_path + 'stock_screener_' +  i.get('dataset').lower() + '.csv')
    
    if generate_data_profile == True:
        log.info('begin pandas profile.html generation')
        generate_docs('ldg_path')

def get_transform(target):
    '''
    Get transform from transform.cfg for target dataset
    #returns a list of dicts with transform logic in format {field:value, fn:value} 
    '''

    lst = [i.get(target) for i in transform['root'] if i.get(target) != None]
    if lst != []: lst = lst[0]

    return lst

def apply_transform(df, transform, target):
    '''
    apply list of tranformstions to dataframe
    '''
    import numpy as np
    log.info('begin transforms for: ' + target)
    
    try: 
        for t in transform: 

            #get function to apply
            fn = t.get('fn')    
            #get reusable function from map_generic_fn if fn starts with $ 
            if fn[0] == '$': fn = map_generic_fn.get(fn)

            #get field or fields to update
            field = t.get('field')
            if field[0] == '[':field = eval(field) 
            #log.info('apply transform: {field, function} ' + str(field) + ' ' + fn)

            #apply transformation
            df[field] = eval(fn)
    except: 
        log.error('error encountered with table:' + str(target) + ' field:' + str(field) + ' fn:' + str(fn) )
    
    log.info('end transforms for: ' + target)
    return 

def normalize_data():
    '''
    Perform preprocessing and copy data to staging area.  
    Preprocessing steps are included in transform.cfg and typically include:
    - metadata validation / data contract
    - preliminary schema normalization
    - data type validation & associated cleansing.  
    '''
    
    ldg_path = env_var.get('ldg_path') 
    stg_path = env_var.get('stg_path')
    
    try:
        #for each dataset in map_landing_dataset_code
        for i in map_landing_dataset_code.get(MODULE):

            #load file for meta contract and data type conversion
            file = ldg_path + i.get('file')
            code = i.get('code')
            df = pd.read_csv(file)

            trns = get_transform(code)
            if trns != []:
                #apply transformation to dataframe
                apply_transform(df,trns,code)

            df.to_csv(stg_path + i.get('file'))
            #TODO Normalize column names, tolwower() with underscores...             
    except:
        log.error('error in normalize_data()')
        
def generate_docs(path):
    '''
    generate pandas_profile.html reports
    '''
    from pandas_profiling import ProfileReport
    
    data_path = env_var.get(path)
    
    try: 
        for i in map_landing_dataset_code[MODULE]:
            file = i.get('file')
            df = pd.read_csv(data_path + file)
            profile = ProfileReport(df, title= 'Profile: ' + file + ' (Landing)')
            fhtml = data_path + 'profile_' + file[0:-4] + '.html'
            fpdf =  data_path + 'profile_' + file[0:-4] + '.pdf'
            profile.to_file(fpdf)
            convert_file_format(fhtml,fpdf)
            
    except:
        log.error('error in generating pandas_profile.html')

def convert_file_format(fromfile, tofile):
    import pdfkit as pk
    pk.from_file(fromfile,tofile)
    

def convert_unit(u):
    if len(u) != 1: 
        ua = u
        u = str(u[-1])
        val = str(ua[0:-1]).replace('.','')
    else:  val = ''
    u=u.lower()

    if u == 'm':
        val += '0000'
    elif u == 'b':
        val += '0000000'
    return val 

In [ ]:
!pip uninstall pdfkit
!pip uninstall weasyprint

In [39]:
import os 

for file in os.listdir('C:\\Users\\SHuds\\training\\finviz-stock-analytics\\docs'):
    print(file)

    data_path = env_var.get('ldg_path')
    fhtml = 'C:\\Users\\SHuds\\training\\finviz-stock-analytics\\docs\\' +   file[0:-4] + 'html'
    fpdf =  'C:\\Users\\SHuds\\training\\finviz-stock-analytics\\docs\\' +   file[0:-4] + 'pdf'
    print(fhtml, fpdf)
    convert_file_format(fhtml,fpdf)


profile_stock_screener_custom.html
C:\Users\SHuds\training\finviz-stock-analytics\docs\profile_stock_screener_custom.html C:\Users\SHuds\training\finviz-stock-analytics\docs\profile_stock_screener_custom.pdf


OSError: No wkhtmltopdf executable found: "b''"
If this file exists please check that this process can read it. Otherwise please install wkhtmltopdf - https://github.com/JazzCore/python-pdfkit/wiki/Installing-wkhtmltopdf

# Download & stage data 

In [17]:
log.info('begin downloading finviz')
#get_data_finviz(generate_data_profile=False)
log.info('end downloading finviz')


log.info('begin finviz preprocessing')
normalize_data()
log.info('end finviz preprocessing')

2020-06-30 17:37:55,626	INFO -- MainProcess -- begin downloading finviz
2020-06-30 17:37:55,626	INFO -- MainProcess -- end downloading finviz
2020-06-30 17:37:55,627	INFO -- MainProcess -- begin finviz preprocessing
2020-06-30 17:37:56,308	INFO -- MainProcess -- begin transforms for: ssfin
2020-06-30 17:37:56,452	INFO -- MainProcess -- Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2020-06-30 17:37:56,453	INFO -- MainProcess -- NumExpr defaulting to 8 threads.
2020-06-30 17:37:56,564	INFO -- MainProcess -- end transforms for: ssfin
2020-06-30 17:37:56,729	INFO -- MainProcess -- begin transforms for: ssovw
2020-06-30 17:37:56,750	INFO -- MainProcess -- end transforms for: ssovw
2020-06-30 17:37:56,853	INFO -- MainProcess -- begin transforms for: sstec
2020-06-30 17:37:56,953	INFO -- MainProcess -- end transforms for: sstec
2020-06-30 17:37:57,139	INFO -- MainProcess -- begin transforms for: sscust
2020-06-30 17:37:57,162	INFO -- MainProc

In [ ]:
def enrich_data_finviz():
    import numpy as np
    
    ENF = err_var.get('no_param')
    stg_path = env_var.get('stg_path')
    
    fown =stg_path + data_var[MODULE].get('stock_ownership',ENF)
    fovr =stg_path + data_var[MODULE].get('stock_overview',ENF)
    key = data_var[MODULE].get('stock_key',ENF)
    
    if (fown==ENF) or (fovr== ENF) or (key == ENF):
        e = 'missing file or key name'
        log_diagnostics('enrich_data_finviz',e,env_var)
        return
    
    #generate additional attibutes
    df_own = pd.read_csv(fown).reset_index()
    df_own.set_index(key,inplace=True)
    df_view = pd.read_csv(fovr).reset_index()
    df_view.set_index(key,inplace=True)
    df = pd.merge(df_own,df_view,how='inner',left_index = True, right_index=True)
    df.reset_index(inplace=True)

    target = 'stg_finviz_summary'            
    transform = get_transform(target) 
    apply_transform(df,transform,target)
    df = df[['Ticker','eps','earnings','P/E','e/p','Outstanding']]

    df.to_csv(stg_path + 'stock_screener_summary.csv')


enrich_data_finviz()